In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import kbcmw_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

def read_json(path, file_name):
    try:
        with open(path+file_name, 'r', encoding='utf-8') as openfile:
            Loaded_file = json.load(openfile)
            print(f"Successfully loaded: {file_name}")
        return Loaded_file
    except Exception as e:
        print(f"Error loading {file_name}: {str(e)}")
        return None

def scrape_article(url, page_key_code):
    try:
        article_content = kbcmw_utils.scrape_kbcmw_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        for i in range(Total_lenght):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_kbcmw_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"kbcmw_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [ ]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tibetcm_ALL_link_ཆེད་བསྒྲིགས་གསར་འགྱུར།.json: 4Total page in tibetcm_ALL_link_ཁམས་ཀྱི་ཁྱབ་ཁོངས།.json: 2
page key name: Page ཁམས་ཀྱི་ཁྱབ་ཁོངས། 1
Page key code: Page ཁམས་ཀྱི་ཁྱབ་ཁོངས། 

page key name: Page ཆེད་བསྒྲིགས་གསར་འགྱུར། 3
Page key code: Page ཆེད་བསྒྲིགས་གསར་འགྱུར། 
Total page in tibetcm_ALL_link_རྒྱལ་ནང་གསར་འགྱུར།.json: 216
page key name: Page རྒྱལ་ནང་གསར་འགྱུར། 215
Page key code: Page རྒྱལ་ནང་གསར་འགྱུར། 
Total page in tibetcm_ALL_link_ཁམས་ཁུལ་གསར་འགྱུར།.json: 264
page key name: Page ཁམས་ཁུལ་གསར་འགྱུར། 263
Page key code: Page ཁམས་ཁུལ་གསར་འགྱུར། 
Total page in tibetcm_ALL_link_རྒྱལ་སྤྱི་གསར་འགྱུར།.json: 33
page key name: Page རྒྱལ་སྤྱི་གསར་འགྱུར། 32
Page key code: Page རྒྱལ་སྤྱི་གསར་འགྱུར། 



  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s][A


  0%|          | 0/7897 [00:00<?, ?it/s]



  0%|          | 0/989 [00:00<?, ?it/s]




  0%|          | 0/6471 [00:00<?, ?it/s]

  3%|▎         | 1/39 [00:06<03:59,  6.30s/it]




  0%|          | 1/6471 [00:06<11:28:58,  6.39s/it]


  0%|          | 1/7897 [00:07<14:29:26,  6.61s/it]



  1%|          | 1/91 [00:08<10:18,  6.87s/it]

  5%|▌         | 2/39 [00:08<01:55,  3.12s/it]




  0%|          | 2/6471 [00:07<5:49:53,  3.25s/it] 


  0%|          | 2/7897 [00:08<7:45:37,  3.54s/it] 



  2%|▏         | 2/91 [00:08<05:24,  3.64s/it]

  8%|▊         | 3/39 [00:08<01:18,  2.18s/it]




  0%|          | 3/6471 [00:07<3:28:11,  1.93s/it]



  1%|          | 9/989 [00:08<08:40,  1.88it/s]


  0%|          | 11/7897 [00:13<2:02:00,  1.08it/s]



  1%|          | 11/989 [00:13<17:02,  1.05s/it]


 13%|█▎        | 12/91 [00:14<01:10,  1.12it/s]



  1%|          | 12/989 [00:13<16:13,  1.00it/s]




  0%

Total Failure in the ཁམས་ཀྱི་ཁྱབ་ཁོངས། article: 0
kbcmw_ALL_content_ཁམས་ཀྱི་ཁྱབ་ཁོངས།.json
Successfully saved: kbcmw_ALL_content_ཁམས་ཀྱི་ཁྱབ་ཁོངས།.json
Total page in tibetcm_ALL_link_རི་ཆུ།.json: 4
page key name: Page རི་ཆུ། 3
Page key code: Page རི་ཆུ། 




  0%|          | 0/120 [00:00<?, ?it/s]



 45%|████▌     | 41/91 [00:35<01:00,  1.21s/it]




  1%|          | 41/6471 [00:34<2:10:42,  1.22s/it]




  1%|          | 42/6471 [00:34<1:42:02,  1.05it/s]



 46%|████▌     | 42/91 [00:35<00:51,  1.05s/it]


 47%|████▋     | 43/91 [00:35<00:38,  1.24it/s]


 48%|████▊     | 44/91 [00:36<00:29,  1.57it/s]


  1%|          | 43/7897 [00:35<1:47:24,  1.22it/s]




  1%|          | 43/6471 [00:35<1:39:50,  1.07it/s]


 49%|████▉     | 45/91 [00:36<00:25,  1.83it/s]



  4%|▍         | 43/989 [00:36<16:40,  1.06s/it]




 51%|█████     | 46/91 [00:36<00:20,  2.22it/s]


  1%|          | 45/7897 [00:36<1:10:54,  1.85it/s]




 52%|█████▏    | 47/91 [00:36<00:16,  2.61it/s]



  4%|▍         | 44/989 [00:36<14:07,  1.12it/s]




 53%|█████▎    | 48/91 [00:37<00:14,  3.06it/s]



  5%|▍         | 46/989 [00:36<08:32,  1.84it/s]


  1%|          | 46/7897 [00:37<1:12:45,  1.80it/s]


  1%|          | 47/7897 [00:37<57:10,  2.29it/s]  



  5%|▍ 

Total Failure in the ཆེད་བསྒྲིགས་གསར་འགྱུར། article: 0
kbcmw_ALL_content_ཆེད་བསྒྲིགས་གསར་འགྱུར།.json
Successfully saved: kbcmw_ALL_content_ཆེད་བསྒྲིགས་གསར་འགྱུར།.json
Total page in tibetcm_ALL_link_གནའ་ཤུལ།.json: 3
page key name: Page གནའ་ཤུལ། 2
Page key code: Page གནའ་ཤུལ། 


...


 82%|████████▏ | 2150/2635 [26:00<03:41,  2.19it/s]




 43%|████▎     | 2763/6471 [33:39<51:14,  1.21it/s]




 82%|████████▏ | 2151/2635 [26:01<04:24,  1.83it/s]


 82%|████████▏ | 2152/2635 [26:01<03:53,  2.07it/s]




 43%|████▎     | 2765/6471 [33:41<48:40,  1.27it/s]


 35%|███▌      | 2784/7897 [33:41<1:25:52,  1.01s/it]




 43%|████▎     | 2766/6471 [33:41<37:28,  1.65it/s]




 82%|████████▏ | 2153/2635 [26:02<04:22,  1.83it/s]




 82%|████████▏ | 2154/2635 [26:03<04:19,  1.85it/s]


 35%|███▌      | 2785/7897 [33:42<1:21:15,  1.05it/s]


 35%|███▌      | 2787/7897 [33:42<48:21,  1.76it/s]  


 35%|███▌      | 2788/7897 [33:42<40:04,  2.13it/s]


 82%|████████▏ | 2155/2635 [26:05<08:55,  1.12s/it]


 35%|███▌      | 2791/7897 [33:45<54:54,  1.55it/s]




 43%|████▎     | 2770/6471 [33:45<1:01:26,  1.00it/s]




 43%|████▎     | 2771/6471 [33:45<52:26,  1.18it/s]  


 82%|████████▏ | 2156/2635 [26:06<08:59,  1.13s/it]




 82%|████████▏ | 2159/2635 [26:07<04:20,  1.83it/s]



Total Failure in the སྙན་རྩོམ། article: 3
kbcmw_ALL_content_སྙན་རྩོམ།.json





 41%|████▏     | 3265/7897 [39:29<1:17:10,  1.00it/s]




Successfully saved: kbcmw_ALL_content_སྙན་རྩོམ།.json


 41%|████▏     | 3267/7897 [39:29<47:02,  1.64it/s]  


 41%|████▏     | 3268/7897 [39:30<38:56,  1.98it/s]


 41%|████▏     | 3269/7897 [39:30<31:28,  2.45it/s]




 50%|█████     | 3247/6471 [39:30<34:52,  1.54it/s]


 41%|████▏     | 3270/7897 [39:30<32:11,  2.40it/s]




 50%|█████     | 3248/6471 [39:30<31:15,  1.72it/s]


 41%|████▏     | 3271/7897 [39:30<29:50,  2.58it/s]




 50%|█████     | 3249/6471 [39:30<30:01,  1.79it/s]


 41%|████▏     | 3272/7897 [39:31<37:13,  2.07it/s]




 50%|█████     | 3250/6471 [39:31<35:32,  1.51it/s]




 50%|█████     | 3251/6471 [39:33<55:05,  1.03s/it]


 41%|████▏     | 3273/7897 [39:34<1:34:28,  1.23s/it]




 50%|█████     | 3252/6471 [39:34<51:57,  1.03it/s]




 50%|█████     | 3253/6471 [39:35<50:08,  1.07it/s]




 50%|█████     | 3255/6471 [39:35<28:57,  1.85it/s]


 41%|████▏     | 3274/7897 [39:35<1:35:19,  1.24s/it]




 50%|█████     | 3256/6471 [39:36<29:04,  1.84it/s]


 41%|████▏     | 3275/7897 [39:36<1:27:37,  1.14s/it]


 4

In [ ]:
7146/7897 [5:15:39<4:41:57, 22.53s/it]

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files